<h1> Title: Spam Detection</h1>

<strong>Overview: In this notebook, I aim to detect spam comments</strong><br>
In this notebook, it covers:<br>
1.0 Heuristic Labeling<br>
2.0 Zero-Shot using API<br>
3.0 Pre-trained model

In [1]:
import re
import pandas as pd

In [2]:
df = pd.read_csv('Datasets/comments.csv',nrows=100)
df["textOriginal"]

0     The, uh, *shape* of the containers is somethin...
1           And with perfect people like you â¤ï¸ðŸŒ¸
2     Please don't call me sir😅😅 best part is you re...
3     Lol All you need is to put your hair in two po...
4                         It's "for the record" by Ooyy
                            ...                        
95                                                   ??
96    Omg this is the cutest ever well done and I mi...
97    i didn't like the dry down of carlisle, i have...
98        KJo should cast him opposition Ranveer Singh.
99    Always at work when you post 🤦🏾‍♀️ going to ha...
Name: textOriginal, Length: 100, dtype: object

## 1.0 Heuristic Labeling (Rule-based Bootstrapping)
- Create a set of rules to automatically label

Strengths: 
- Fast to implement.
- Can capture obvious spam patterns (URLs, “follow me,” mass tagging).

Weaknesses:
- Misses subtle spam 
- May mislabel genuine comments with links

In [16]:
def heuristic_label(comment: str) -> bool:
    """
    Returns True if the comment is considered spam based on heuristic rules,
    otherwise False.
    """
    # Normalize text
    text = comment.lower().strip()
    
    # Rule 1: Suspicious keywords / links 
    spam_keywords = [
        "http://", "https://", "www.",
        "check out my", "follow me",
        "@@", "free", "subscribe", "visit my",
        "win money", "free gift", "dm for promo", "subscribe my channel"
    ]
    if any(keyword in text for keyword in spam_keywords):
        return True
    
    # Rule 2: Regex patterns
    regex_patterns = [
        r"[a-z0-9]{20,}",   # Long alphanumeric strings (promo codes etc.)
        r"([!@#$%^&*()_+=\-{}\[\]:;\"'<>,.?/\\|]){5,}",  # Excessive special chars
        r"http[s]?://"      # URLs
    ]
    for pattern in regex_patterns:
        if re.search(pattern, text):
            return True
    
    # Rule 3: Very short (1–3 words) generic comments
    if len(text.split()) <= 2:
        return True
    
    return False

In [17]:
# Apply heuristic labeling
df["spam"] = df["textOriginal"].apply(heuristic_label)

# Print only spam comments
spam_comments = df[df["spam"] == True]
print(spam_comments["textOriginal"])

6                                               Alright
16                             ðŸ˜®ðŸ˜®ðŸ˜®ðŸ˜®ðŸ˜®ðŸ˜®
19                                                ❤❤❤❤❤
20                                                 ❤❤❤❤
23                                                 Bhhh
25                                    â¤ðŸ˜‚â¤ðŸ˜‚â¤
27                                                   💥💥
28                                              True😂😂❤
30                                              Niceâ¤
31                                     Elegant makeover
37                                                  â¤
40                                                  ❤❤❤
45                                          Reuploaded😂
51                                       Superb content
53                                              Hii sis
55                          👍🏽👍🏽👍🏽👍🏽❤️💙💛💛🧡so cool👍🏽👍🏽👍🏽
56                                           Thankiiies
59                                         ðŸ˜‚ð

## 2.0 Zero-Shot with LLM API (Pseudo-labeling)
- Use a large language model (LLM) API or a to generate "pseudo-labels"

Strengths:
- Can handle nuanced cases heuristics miss.
- No need to manually annotate everything.

Weaknesses:
- Expensive at scale.
- Quality depends on prompt + LLM.

In [ ]:
from openai import OpenAI

# Initialize client
client = OpenAI(api_key="") # set your API key as environment variable: OPENAI_API_KEY / OPENROUTER_API_KEY

def zero_shot(comment: str) -> str:
    """
    Use an LLM to pseudo-label a comment as 'spam' or 'not spam'.
    """
    prompt = f"""
    You are a spam detection system.
    Classify the following comment as either 'spam' or 'not spam'.
    Only output 'spam' or 'not spam'.

    Comment: "{comment}"
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # /"gpt-4o-mini"
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    label = response.choices[0].message.content.strip().lower()
    return label if label in ["spam", "not spam"] else "not spam"

In [ ]:
# Apply Zero shot
df["spam"] = df["textOriginal"].apply(zero_shot)

# Filter only spam
spam_comments = df[df["spam"] == "spam"]

print(spam_comments[["textOriginal", "spam"]])

                                         textOriginal  spam
10                       Share it. Share it. Share it  spam
72  hey bro..your really handsome lovely..would li...  spam
83                                         ðŸ™‹âœŒï¸  spam
90                           ðŸ¥°ðŸ¥°ðŸ¥°ðŸ¥°ðŸ¥°ðŸ’‹  spam


## 3.0 Pre-trained Spam Detection Model

Strengths:
- Already optimized for spam/ham classification.
- Usually faster & cheaper than LLM APIs.

Weaknesses:
- May not generalize perfectly to video comment domain (different slang, formats).

<h3>Label Description</h3>

1. **Noise**: Gibberish at the zero level where even the different constituents of the input phrase (words) do not hold any meaning independently.
    - For example: dfdfer fgerfow2e0d qsqskdsd djksdnfkff swq.

2. **Word Salad**: Gibberish at level 1 where words make sense independently, but when looked at the bigger picture (the phrase) any meaning is not depicted.
    - For example: 22 madhur old punjab pickle chennai

3. **Mild gibberish**: Gibberish at level 2 where there is a part of the sentence that has grammatical errors, word sense errors, or any syntactical abnormalities, which leads the sentence to miss out on a coherent meaning.
    - For example: Madhur study in a teacher

4. **Clean**: This category represents a set of words that form a complete and meaningful sentence on its own.
    - For example: I love this website

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load model and tokenizer from Hugging Face Hub
model = AutoModelForSequenceClassification.from_pretrained("madhurjindal/autonlp-Gibberish-Detector-492513457")
tokenizer = AutoTokenizer.from_pretrained("madhurjindal/autonlp-Gibberish-Detector-492513457")

# Predicts whether a list of comments is spam using a pre-trained model.
def pretrained_model(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_label_id = probabilities.argmax().item()
    
    return model.config.id2label[predicted_label_id]

df["spam"] = df["textOriginal"].apply(pretrained_model)

print(df[["textOriginal", "spam"]])

                                         textOriginal            spam
0   The, uh, *shape* of the containers is somethin...  mild gibberish
1         And with perfect people like you â¤ï¸ðŸŒ¸      word salad
2   Please don't call me sir😅😅 best part is you re...  mild gibberish
3   Lol All you need is to put your hair in two po...           clean
4                       It's "for the record" by Ooyy           clean
..                                                ...             ...
95                                                 ??      word salad
96  Omg this is the cutest ever well done and I mi...  mild gibberish
97  i didn't like the dry down of carlisle, i have...  mild gibberish
98      KJo should cast him opposition Ranveer Singh.  mild gibberish
99  Always at work when you post 🤦🏾‍♀️ going to ha...  mild gibberish

[100 rows x 2 columns]


In [27]:
print(df["spam"].value_counts())

spam
mild gibberish    46
clean             25
noise             17
word salad        12
Name: count, dtype: int64


In [32]:
# Filter only rows predicted as clean
clean_df = df[df["spam"] == "clean"]

print(clean_df[["textOriginal", "spam"]])

                                         textOriginal   spam
3   Lol All you need is to put your hair in two po...  clean
4                       It's "for the record" by Ooyy  clean
7      Please show us what these are like in the sun.  clean
8                   This is a great point, thank you!  clean
9   Pretty brave to try it anyway after dreaming t...  clean
10                       Share it. Share it. Share it  clean
17  Everything looks great except the red hat on b...  clean
18  Can you comb or brush through it without it co...  clean
24  Is there a semi permanent hair dye out there t...  clean
29       Does anyone know the title of the intro song  clean
31                                   Elegant makeover  clean
36           I'm so glad it worked. It looks so cool.  clean
38  I thought she was taking a screenshot in the w...  clean
47  You could but some of the product would defini...  clean
49  Would totally depend on the style of the outfi...  clean
50  You're so pretty Ais

In [33]:
# Filter only rows predicted as noise
noise_df = df[df["spam"] == "noise"]

print(noise_df[["textOriginal", "spam"]])

                                         textOriginal   spam
12  I'm so happy thanks for the song â¤â¤â¤â¤ð...  noise
13                                Muito preciosa ðŸ’Ž  noise
14  Como carajo usas una gorra pobre fracasado? Ja...  noise
21               Bhaii lacto calamine pe video bna do  noise
23                                               Bhhh  noise
25                                  â¤ðŸ˜‚â¤ðŸ˜‚â¤  noise
32                           Love from India ðŸ‡®ðŸ‡³  noise
33   iw ye hee hee cade i❤dvv birwe😂g🎉isod I love you  noise
34  Baal to mere bhi jhad rahe hai...\n.\n.\nPar m...  noise
53                                            Hii sis  noise
59                                       ðŸ˜‚ðŸ˜‚ðŸ‘  noise
69                             Handsome âŽ chapriâœ…  noise
70                                             â¤â¤  noise
71  C'est une blague tous ces avis négatifs ? C'es...  noise
83                                         ðŸ™‹âœŒï¸  noise
90                      

In [36]:
# Filter only rows predicted as word salad
salad_df = df[df["spam"] == "word salad"]

print(salad_df[["textOriginal", "spam"]])

                                         textOriginal        spam
1         And with perfect people like you â¤ï¸ðŸŒ¸  word salad
61                                   Her smile tho ❤🤗  word salad
62                                             Good ❤  word salad
64                            How old are you sir â¤  word salad
66                                            Awesome  word salad
73  @@SunitaRawat-tz4nbwhen i went to go look on t...  word salad
74                                    Coming soon. :)  word salad
75                                           Will do!  word salad
82                         This blonde hair though!!❤  word salad
86                                          Beautiful  word salad
88                              Nope. English upload.  word salad
95                                                 ??  word salad


In [35]:
# Filter only rows predicted as mild gibberish
gibberish_df = df[df["spam"] == "mild gibberish"]

print(gibberish_df[["textOriginal", "spam"]])

                                         textOriginal            spam
0   The, uh, *shape* of the containers is somethin...  mild gibberish
2   Please don't call me sir😅😅 best part is you re...  mild gibberish
5     All that to look just above average teanage boy  mild gibberish
6                                             Alright  mild gibberish
11   What lash searums do you recommend that is cheap  mild gibberish
15                                 Plz toner recipe 😊  mild gibberish
16                           ðŸ˜®ðŸ˜®ðŸ˜®ðŸ˜®ðŸ˜®ðŸ˜®  mild gibberish
19                                              ❤❤❤❤❤  mild gibberish
20                                               ❤❤❤❤  mild gibberish
22  I still dont understand should I apply wax fir...  mild gibberish
26  @sharshot adkari Already plenty videos on that...  mild gibberish
27                                                 💥💥  mild gibberish
28                                            True😂😂❤  mild gibberish
30                  